In [1]:
from pymorphy2 import MorphAnalyzer
from gensim.models import KeyedVectors
from time import time
import numpy as np
import pandas as pd

load model

In [2]:
model = KeyedVectors.load_word2vec_format('word_to_vec.bin', binary=True)

In [3]:
morph = MorphAnalyzer()
def handle_word(word):
    word = word.lower().strip('(.,!?:;\'"«»—-–)')
    p = morph.parse(word)[0]
    return p.normal_form
def get_average_vector(words):
    vectors = []
    for word in words:
        parsed = handle_word(word)
        if parsed in model:
            vectors.append(model[parsed])
        else:
            vectors.append(np.zeros(100))
    return np.mean(vectors, axis=0)


print(get_average_vector(['привет', 'как', 'дела']))

[ 0.01319534 -0.44012567 -0.24828333 -0.30982634  0.44310102  0.18418433
  0.12236699 -0.06977067 -0.22141068  0.33174     0.04571233  0.4523003
  0.05751466 -0.25541267 -0.10298267  0.32320166  0.28154334  0.05418434
  0.45345068  0.20138998 -0.389215    0.384002    0.34564066 -0.25499168
  0.16727601  0.03322067  0.00858967  0.066601   -0.05029468 -0.19079167
  0.04519533 -0.027845   -0.257506    0.01388633 -0.04997567 -0.03241967
  0.014428   -0.11890599  0.029228   -0.34692034 -0.26247802  0.298645
  0.3107127  -0.14327334 -0.15045999  0.129627    0.20942901 -0.05834166
  0.17391133 -0.16441034  0.12307634 -0.14922099 -0.061433    0.22274001
  0.13244967  0.02664267  0.07971966  0.3478087  -0.11845666 -0.29305732
 -0.074766   -0.02340234 -0.02104733 -0.14968766  0.07986266  0.24631031
  0.020461   -0.23431699  0.6417493  -0.041198    0.48819467  0.04603934
  0.02268433 -0.2872503  -0.005793   -0.10669366 -0.32312167  0.13104033
  0.01454767 -0.22922999  0.085271   -0.13496734  0.05

#### check model 

In [4]:
# all_words = list(set([word for text in data['uu_usl_name'].values[:10000] for word in text.split()]))
# print(len([handle_word(word) for word in all_words if handle_word(word) in model]))

NameError: name 'data' is not defined

In [5]:
data = pd.read_csv('dataset.csv', sep=',')
data.head()

,uu_usl_name,CLASS_ID,SERVICE_CLASS_CONFIRMED,CNT_DIST_USL_BY_CLASS,CNT_NUM_USL
0,АМБУЛАТОРНАЯ ПОМОЩЬ,1000,NaN,12614,812414
1,АМБУЛАТОРНАЯ ПОМОЩЬ,1292,NaN,777,812414
2,КОНСУЛЬТАТИВНО-ДИАГНОСТИЧЕСКАЯ ПОМОЩЬ,1000,NaN,12614,740580
3,Сбербанк Премьер,3200,1.0,3974,489254
4,Сбербанк Премьер,8350,1.0,104,489254


In [6]:
#clear data
columns_to_delete = ['SERVICE_CLASS_CONFIRMED', 'CNT_DIST_USL_BY_CLASS', 'CNT_NUM_USL']
data = data.drop(columns_to_delete, axis=1)
data = data[data.CLASS_ID != 9000]

In [8]:
pre_X = [get_average_vector(text.split()) for text in data['uu_usl_name'].values]

In [7]:

X = pd.DataFrame(pre_X, dtype=float).fillna(0)
X.head()

NameError: name 'pre_X' is not defined

In [ ]:
y = data['CLASS_ID'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier(
    n_estimators=1,
    max_depth=10,
    random_state=0,
    n_jobs=-1
)



In [ ]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=1, n_jobs=-1, random_state=0)

In [ ]:
pred_X = clf.predict(X_test)
accuracy_score(y_test, pred_X)